In [1]:
from __future__ import division

import os
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [30]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.AtomPairs.Pairs import GetAtomPairFingerprint

In [3]:
txt_dir = "chembl_source"

In [4]:
# read all chembl bioactivity records
chembl = pd.read_csv(os.path.join(txt_dir, "inhibitor_2017_06_08.csv"), delimiter="\t")
chembl.shape

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,3,6,7,8,9,11,14,16,19,23,27,31,34,35,38,44,48,50,52,53,54,55,56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1235867, 59)

In [5]:
# Remove records has no canonical smiles
m = chembl["CANONICAL_SMILES"].isnull()
chembl = chembl[~m]
chembl.shape

(1230260, 59)

In [21]:
# save inhibitors' smiles and apfp
smiles = chembl[["CMPD_CHEMBLID", "CANONICAL_SMILES"]].copy()
smiles.drop_duplicates(subset="CMPD_CHEMBLID", inplace=True)
smiles.set_index(keys="CMPD_CHEMBLID", drop=True, inplace=True)
smiles.to_csv(txt_dir + "/inhibitor_smiles.csv")

In [31]:
def dict_2_str(d):
  keylist = d.keys()
  keylist.sort()
  kv_list = ["{}: {}".format(k, d[k]) for k in keylist] 
  return ", ".join(kv_list)

apfp_file = open(txt_dir + "/inhibitor_apfp.csv", "w")
for id_, row in smiles.iterrows():
    m = Chem.MolFromSmiles(row.values[0])
    if m is None:
        print id_
        continue
    apfps = GetAtomPairFingerprint(Chem.RemoveHs(m)).GetNonzeroElements()
    apfp_file.write("%s\t{%s}\n" % (id_, dict_2_str(apfps)))
apfp_file.close()

CHEMBL1161633
CHEMBL2097021
CHEMBL471869
CHEMBL1161635
CHEMBL181124
CHEMBL1161637
CHEMBL181880
CHEMBL3593577
CHEMBL450200
CMPD_CHEMBLID
CHEMBL450642
CHEMBL2205792
CHEMBL2205793
CHEMBL490121
CHEMBL523281
CHEMBL463327
CHEMBL522826
CHEMBL2205790
CHEMBL495469
CHEMBL2205791
CHEMBL492602
CHEMBL2205788
CHEMBL2205787
CHEMBL452133
CHEMBL2205785
CHEMBL508580
CHEMBL508803
CHEMBL2205789
CHEMBL2205786
CHEMBL493431
CHEMBL2087763
CHEMBL2087764
CHEMBL2179461
CHEMBL2179458
CHEMBL2179464
CHEMBL2179462
CHEMBL2179459
CHEMBL2179463
CHEMBL1083554
CHEMBL2179460
CHEMBL3327018


In [6]:
# calculate some molecules's weight
def molwt(x):
    try:
        value = Chem.Descriptors.MolWt(Chem.MolFromSmiles(x))
    except:
        value = np.nan
    return value

m = chembl["MOLWEIGHT"].isnull()
chembl.loc[m, "MOLWEIGHT"] = chembl.loc[m, "CANONICAL_SMILES"].apply(molwt)

In [8]:
# remove molecules that has no "MOLWEIGHT"
m = chembl["MOLWEIGHT"].isnull()
chembl = chembl[~m]
chembl.shape

(1223639, 59)

In [32]:
# pick out inhibitor records
inhibitor = chembl[chembl["STANDARD_TYPE"].isin(["IC50", "Ki", "EC50"])]

# inhibitor records: all IC50, a part of Ki and EC50 with "inhibit" in "DESCRIPTION"
m0 = inhibitor["STANDARD_TYPE"].isin(["IC50"]) 
m1 = inhibitor["STANDARD_TYPE"].isin(["Ki", "EC50"]) 
m2 = inhibitor["DESCRIPTION"].apply(lambda x: "inhibit" in x.lower())
m = m0 | (m1 & m2)

inhibitor = inhibitor[m]
inhibitor.shape

(835299, 59)

In [33]:
# some records without "STANDARD_VALUE" should be cleared away
m = inhibitor["STANDARD_VALUE"].isnull()
inhibitor = inhibitor[~m]
inhibitor.shape

(716442, 59)

In [34]:
inhibitor["DATA_VALIDITY_COMMENT"].value_counts()

Outside typical range            26411
Potential transcription error      378
Non standard unit for type         370
Manually validated                 163
Name: DATA_VALIDITY_COMMENT, dtype: int64

In [35]:
# some records with abnormal data also should be cleared away
#error_comment = ["Outside typical range", "Non standard unit for type", "Potential transcription error"]
error_comment = ["Outside typical range"]
m = inhibitor["DATA_VALIDITY_COMMENT"].isin(error_comment)
inhibitor = inhibitor[~m]
inhibitor.shape

(690031, 59)

In [36]:
# correct some STANDARD_UNITS
m = inhibitor["STANDARD_UNITS"].isin(["/uM"])
inhibitor.loc[m, "STANDARD_VALUE"] = inhibitor.loc[m, "STANDARD_VALUE"].astype(float).values * 1000
inhibitor.loc[m, "STANDARD_UNITS"] = "nM"

m = inhibitor["STANDARD_UNITS"].isin(["/nM", "ug nM-1", "Ke nM-1"])
inhibitor.loc[m, "STANDARD_UNITS"] = "nM"

m = inhibitor["STANDARD_UNITS"].isin(["ug.mL-1"])
inhibitor.loc[m, "STANDARD_VALUE"] = inhibitor.loc[m, "STANDARD_VALUE"].astype(float) / inhibitor.loc[m, "MOLWEIGHT"].astype(float) * 10**6
inhibitor.loc[m, "STANDARD_UNITS"] = "nM"

m = inhibitor["STANDARD_UNITS"].isin(["nM"])
inhibitor = inhibitor[m]
inhibitor.shape

(689725, 59)

In [37]:
# remove duplicates
m = inhibitor["POTENTIAL_DUPLICATE"].fillna(0).astype(int) == 0
inhibitor = inhibitor[m]
inhibitor.shape

(662788, 59)

In [39]:
inhibitor.to_csv(txt_dir + "/inhibitor_clean_2017_06_08.csv", index=False)

In [ ]:
# judge a record's clf label
def is_pos(row):
  r = row["RELATION"]
  v = np.float32(row["STANDARD_VALUE"])
  if r == "<" or r == "<=":
    return 1 if v <= 10000 else np.nan
  elif r == ">" or r == ">=":
    return -1 if v >= 10000 else np.nan
  elif r == "=":
    return 1 if v <= 10000 else -1
  else:
    return np.nan

In [89]:
inhibitor["CLF_LABEL"] = inhibitor.apply(is_pos, axis=1)
inhibitor = inhibitor[~inhibitor["CLF_LABEL"].isnull()]
inhibitor.loc[:, "YEAR"] = inhibitor.loc[:, "YEAR"].astype(float)

In [131]:
# group
grouped = inhibitor.groupby(by=["TARGET_CHEMBLID", "PREF_NAME", "CMPD_CHEMBLID"], as_index=False)
# judge one molecule's label by the average label
clf_label = grouped[["CLF_LABEL", "YEAR"]].mean()
clf_label

,TARGET_CHEMBLID,PREF_NAME,CMPD_CHEMBLID,CLF_LABEL,YEAR
0,CHEMBL1075092,Glycine receptor subunit alpha-3,CHEMBL1092618,-1.0,2010.0
1,CHEMBL1075092,Glycine receptor subunit alpha-3,CHEMBL1092619,-1.0,2010.0
2,CHEMBL1075092,Glycine receptor subunit alpha-3,CHEMBL1093582,-1.0,2010.0
3,CHEMBL1075092,Glycine receptor subunit alpha-3,CHEMBL1093848,-1.0,2010.0
4,CHEMBL1075092,Glycine receptor subunit alpha-3,CHEMBL2398350,-1.0,2013.0
5,CHEMBL1075092,Glycine receptor subunit alpha-3,CHEMBL2398352,-1.0,2013.0
6,CHEMBL1075092,Glycine receptor subunit alpha-3,CHEMBL464651,1.0,2010.0
7,CHEMBL1075097,Arginase-1,CHEMBL1099169,1.0,2010.0
8,CHEMBL1075101,G-protein coupled receptor 81,CHEMBL3714817,1.0,NaN
9,CHEMBL1075101,G-protein coupled receptor 81,CHEMBL3714879,1.0,NaN


In [105]:
clf_label.to_csv(txt_dir + "/inhibitor_clf_label.csv")

In [144]:
cancer_approved_target = ["CHEMBL279", "CHEMBL203", "CHEMBL333", "CHEMBL325", "CHEMBL267", "CHEMBL2842"]
cancer_clinical_target = ["CHEMBL340", "CHEMBL4005", "CHEMBL332"]

In [158]:
for target in cancer_approved_target + cancer_clinical_target:
    df = clf_label[clf_label["TARGET_CHEMBLID"] == target]
    df.to_csv(txt_dir + "/%s_clf_label.csv" % target, index=False)